In [58]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime as dt
from IPython.core.display import display, HTML
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 200
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
# Reading in FCC wisp data
filepath = "../data/bb_data.csv"
wisp_df = pd.read_csv(filepath)

In [13]:
# Reading in population densities
filepath = "../data/us2016.csv"
pop_df = pd.read_csv(filepath)

In [21]:
# Pennsylvania population density
pa_pop_df = pop_df[pop_df['stateabbr'] == 'PA']

# Pennsylvania WISPs
pa_wisp_df = wisp_df[wisp_df['State']=='PA']

In [70]:
'''
Population information:

hu = housing units
hh = households
pop = population

'''
cols = ['stateabbr','block_fips','hu2016','hh2016','pop2016']
pa_pop_df = pa_pop_df[cols]

stateabbr                  PA
block_fips    420010301011000
hu2016                      3
hh2016                      3
pop2016                     6
Name: 8103116, dtype: object

In [139]:
print(len(pa_wisp_df['Census Block FIPS Code'].unique()))
print(pa_wisp_df.columns.values)

421545
['Logical Record Number' 'Provider ID' 'FRN' 'Provider Name' 'DBA Name'
 'Holding Company Name' 'Holding Company Number' 'Holding Company Final'
 'State' 'Census Block FIPS Code' 'Technology Code' 'Consumer'
 'Max Advertised Downstream Speed (mbps)'
 'Max Advertised Upstream Speed (mbps)' 'Business'
 'Max CIR Downstream Speed (mbps)' 'Max CIR Upstream Speed (mbps)']


In [194]:
def get_metrics(raw_df):
    
    # A count of all the unique WISP's in a given census block,  if block isn't included in dataframe, then there are no WISPs in that block
    wisp_count_df = pd.DataFrame({'wisp_count':raw_df.groupby(['Census Block FIPS Code']).size()}).reset_index()
    wisp_count_df.set_index('Census Block FIPS Code', inplace=True)

    # Number of WISPs allowed to serve consumers in the given block
    consumer_df = pd.DataFrame({'consumer_num':raw_df.groupby(['Census Block FIPS Code','Consumer']).size()}).reset_index()
    consumer_df = consumer_df[consumer_df['Consumer']==1]
    consumer_df.set_index('Census Block FIPS Code', inplace=True)

    # Number of WISPs allowed to serve businesses in the given block
    business_df = pd.DataFrame({'business_num':raw_df.groupby(['Census Block FIPS Code', 'Business']).size()}).reset_index()
    business_df = business_df[business_df['Business']==1]
    business_df.set_index('Census Block FIPS Code', inplace = True)

    # Average up/downstream speeds in each census block
    adv_max_down_df = pd.DataFrame({'adv_max_down':raw_df.groupby(['Census Block FIPS Code'])['Max Advertised Downstream Speed (mbps)'].mean()}).reset_index()
    adv_max_down_df.set_index('Census Block FIPS Code', inplace = True)
    
    adv_max_up_df = pd.DataFrame({'adv_max_up':raw_df.groupby(['Census Block FIPS Code'])['Max Advertised Upstream Speed (mbps)'].mean()}).reset_index()
    adv_max_up_df.set_index('Census Block FIPS Code', inplace = True)
    
    cir_max_up_df = pd.DataFrame({'cir_max_up':raw_df.groupby(['Census Block FIPS Code'])['Max CIR Upstream Speed (mbps)'].mean()}).reset_index()
    cir_max_up_df.set_index('Census Block FIPS Code', inplace = True)
    
    cir_max_down_df = pd.DataFrame({'cir_max_down':raw_df.groupby(['Census Block FIPS Code'])['Max CIR Downstream Speed (mbps)'].mean()}).reset_index()
    cir_max_down_df.set_index('Census Block FIPS Code', inplace = True)
    
    #TODO: Merge the 7 df's
    df_list = [wisp_count_df,consumer_df,business_df,adv_max_down_df,adv_max_up_df,cir_max_up_df,cir_max_down_df]
    print('Merging dataframes...')
    for i in df_list:
        if wisp_count_df.equals(i):
            final_df = wisp_count_df
        else:
            final_df = pd.merge(final_df,i,how='outer',left_index=True, right_index=True)
    print('Done merging dataframes!')
    final_df.drop(['Consumer', 'Business'], axis = 1, inplace = True)
    return final_df 

In [195]:
res_df = get_metrics(pa_wisp_df)

Merging dataframes...
Done merging dataframes!


,wisp_count,consumer_num,business_num,adv_max_down,adv_max_up,cir_max_up,cir_max_down
Census Block FIPS Code,,,,,,,
420010301011000,7,6,7.0,38.142857,2.724000,0.471429,2.428571
420010301011001,7,6,7.0,38.142857,2.724000,0.471429,2.428571
420010301011002,8,7,8.0,33.750000,2.479500,0.412500,2.125000
420010301011003,7,6,7.0,40.285714,2.900000,0.471429,2.428571
420010301011004,7,6,7.0,38.857143,2.828571,0.471429,2.428571


In [169]:
res_df.iloc[0]['Census Block FIPS Code_x'].values[0]

420010301011000.0